In [ ]:
import win32evtlog
import sys
from xml.etree import ElementTree as ET
import win32con


server = 'localhost'
logtype = 'Security'

In [ ]:
evt_dict = {win32con.EVENTLOG_AUDIT_FAILURE: 'EVENTLOG_AUDIT_FAILURE',
            win32con.EVENTLOG_AUDIT_SUCCESS: 'EVENTLOG_AUDIT_SUCCESS',
            win32con.EVENTLOG_INFORMATION_TYPE: 'EVENTLOG_INFORMATION_TYPE',
            win32con.EVENTLOG_WARNING_TYPE: 'EVENTLOG_WARNING_TYPE',
            win32con.EVENTLOG_ERROR_TYPE: 'EVENTLOG_ERROR_TYPE'}


def parse_XML_log(event):
    tree = ET.ElementTree(ET.fromstring(event))
    root = tree.getroot()
    ns = "{http://schemas.microsoft.com/win/2004/08/events/event}"
    data = {}
    for eventID in root.findall(".//"):
        if eventID.tag == f"{ns}System":
            for e_id in eventID.iter():
                if e_id.tag == f"{ns}System":
                    pass
                elif e_id.tag == f"{ns}Provider":
                    data["Provider"] = e_id.attrib.get('Name')
                elif e_id.tag == f"{ns}TimeCreated":
                    data["TimeCreated"] = e_id.attrib.get('SystemTime')
                elif e_id.tag == f"{ns}Correlation":
                    data["ActivityID"] = e_id.attrib.get('ActivityID')
                elif e_id.tag == f"{ns}Execution":
                    data["ProcessID"] = e_id.attrib.get('ProcessID')
                    data["ThreadID"] = e_id.attrib.get('ThreadID')
                elif e_id.tag == f"{ns}EventType":
                    if not e_id.text in evt_dict.keys():
                        data['EventType'] = "unknown"
                    else:
                        data['EventType'] = str(evt_dict[e_id.text])
                else:
                    att = e_id.tag.replace(f"{ns}", "")
                    data[att] = e_id.text

        if eventID.tag == f"{ns}EventData":
            for attr in eventID.iter():
                if attr.tag == f'{ns}Data':
                    if attr.get('Name') is None:
                        data["Data"] = attr.text
                    else:
                        data[attr.get('Name')] = attr.text
                elif attr.tag == f'{ns}Binary':
                    data["Binary"] = attr.text

    return data

In [ ]:
event_context = { "info": "this object is always passed to your callback" }
# Event log source to listen to
event_source = 'application'

def new_logs_event_handler(reason, context, evt):
    """
    Called when new events are logged.

    reason - reason the event was logged?
    context - context the event handler was registered with
    evt - event handle
    """
    # Just print some information about the event
    # print ('reason', reason, 'context', context, 'event handle', evt)

    event = win32evtlog.EvtRender(evt, win32evtlog.EvtRenderEventXml)
    result = " ".join(l.strip() for l in event.splitlines())
    print(result)
    with open('xml_logs.txt', 'a') as file:
        file.write(result)
        file.write('\n')
    # parse_XML_log(event=event)
    print(' - ')

    # Make sure all printed text is actually printed to the console now
    sys.stdout.flush()
    return 0

In [ ]:


# Subscribe to future events
subscription1 = win32evtlog.EvtSubscribe(event_source, win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription2 = win32evtlog.EvtSubscribe('system', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription3 = win32evtlog.EvtSubscribe('Security', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)


In [ ]:
data = []
with open("xml_logs.txt", "r+") as file1:
    lines = file1.readlines()   
    for line in lines:              
        data.append(parse_XML_log(line))   
        
data


In [ ]:
for list in data:
    print(list)

In [ ]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Windows_Event_Logs']
collection = db['mycollection']


# Insert the list into the collection

collection.insert_many(data)

# Print the inserted documents
for doc in collection.find():
    print(doc)

In [ ]:
import pyuac

def main():
    print("Do stuff here that requires being run as an admin.")
    # The window will disappear as soon as the program exits!
    input("Press enter to close the window. >")

if __name__ == "__main__":
    if not pyuac.isUserAdmin():
        print("Re-launching as admin!")
       
        pyuac.runAsAdmin()
        input("Press enter to close the window. >")
    else:        
        main()  # Already an admin here.

In [ ]:
import json
import xmltodict

def xml_to_json(xml_string):
    xml_dict = xmltodict.parse(xml_string)
    print(xml_dict)
    json_data = json.dumps(xml_dict)
    return json_data

In [ ]:
js = xml_to_json(lines[0])
js

In [ ]:
with open('test.json', 'w') as f:
    f.write(js)